In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
# MPC import
import numpy as np
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = ...
H = ...
x0 = ...  # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);

### MPC_Control_VelX 1 step OL

In [83]:
from LinearMPC.MPCControl_xvel import MPCControl_xvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 1
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.1, 0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_x = MPCControl_xvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_x.get_u(x0)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

u0 =  [0.02622129]


### MPC_Control_VelY 1 step OL

In [95]:
from LinearMPC.MPCControl_yvel import MPCControl_yvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 1
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.1, 0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_y = MPCControl_yvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_y.get_u(x0)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

u0 =  [-0.02622129]


c:\Users\loeff\miniconda3\envs\mpc2025\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


### MPC_Control_VelZ 1 step OL

In [102]:
from LinearMPC.MPCControl_zvel import MPCControl_zvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 1
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.1, 0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_z = MPCControl_zvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_z.get_u(x0)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

# Debug: check solver status


u0 =  [66.46813216]


### MPC_Control_ROLL 1 step OL

In [97]:
from LinearMPC.MPCControl_roll import MPCControl_roll
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 1
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.1, 0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_roll = MPCControl_roll(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_roll.get_u(x0)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

# Debug: check solver status


u0 =  [-3.4741594e-28]


c:\Users\loeff\miniconda3\envs\mpc2025\Lib\site-packages\cvxpy\reductions\solvers\solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


### Open-Loop trajectory

In [ ]:
Ts = 0.05
t0 = 0
x0 = np.array([0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0]) 
rocket = Rocket(Ts,model_params_filepath=rocket_params_path)
H = 5 # MPC horizon in seconds
mpc_ol = MPCVelControl().new_controller(rocket, Ts, H)
u, x_ol, u_ol, t_ol = mpc_ol.get_u(t0, x0)

In [117]:
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_ol[:-1], x_ol[:,:-1], u_ol);


AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=399, step=2), IntSlider(value=0…

### Close-loop trajectory

In [166]:

H = 5 # MPC horizon in seconds
Ts = 0.05
t0 = 0
x0 = np.array([0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0]) 
rocket = Rocket(Ts=Ts,model_params_filepath=rocket_params_path)
mpc_cl = MPCVelControl().new_controller(rocket, Ts, H)
u, x_ol, u_ol, t_ol = mpc_cl.get_u(t0, x0)

In [167]:
sim_time = 3 # Simulation length
#x1 = rocket.simulate_step(x0, Ts, u, method='nonlinear')
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(mpc_cl, sim_time, H, x0, method='linear')

Simulating time 0.00: 
Simulating time 0.05: 
Simulating time 0.10: 
Simulating time 0.15: 
Simulating time 0.20: 
Simulating time 0.25: 
Simulating time 0.30: 
Simulating time 0.35: 
Simulating time 0.40: 
Simulating time 0.45: 
Simulating time 0.50: 
Simulating time 0.55: 
Simulating time 0.60: 
Simulating time 0.65: 
Simulating time 0.70: 
Simulating time 0.75: 
Simulating time 0.80: 
Simulating time 0.85: 
Simulating time 0.90: 
Simulating time 0.95: 
Simulating time 1.00: 
Simulating time 1.05: 
Simulating time 1.10: 
Simulating time 1.15: 
Simulating time 1.20: 
Simulating time 1.25: 
Simulating time 1.30: 
Simulating time 1.35: 
Simulating time 1.40: 
Simulating time 1.45: 
Simulating time 1.50: 
Simulating time 1.55: 
Simulating time 1.60: 
Simulating time 1.65: 
Simulating time 1.70: 
Simulating time 1.75: 
Simulating time 1.80: 
Simulating time 1.85: 
Simulating time 1.90: 
Simulating time 1.95: 
Simulating time 2.00: 
Simulating time 2.05: 
Simulating time 2.10: 
Simulating 

In [168]:
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol);


AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=59, step=2), IntSlider(value=0,…